In [1]:
import numpy as np
import timeit
import pandas as pd
from sklearn import decomposition
import fbpca
import torch
from scipy import linalg

In [2]:
def simple_randomized_svd(M, k=10):
    m, n = M.shape
    transpose = False
    if m < n:
        transpose = True
        M = M.T
    rand_matrix = np.random.normal(size=(M.shape[1], k))  # short side by k
    Q, _ = np.linalg.qr(M @ rand_matrix, mode='reduced')  # long side by k
    smaller_matrix = Q.T @ M                              # k by short side
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat
    
    if transpose:
        return V.T, s.T, U.T
    else:
        return U, s, V

In [3]:
def simple_randomized_torch_svd(M, k=10):
    B = torch.tensor(M).cuda(0)
    m, n = B.size()
    transpose = False
    if m < n:
        transpose = True
        B = B.transpose(0, 1).cuda(0)
        m, n = B.size()
    rand_matrix = torch.rand((n,k), dtype=torch.double).cuda(0)  # short side by k
    Q, _ = torch.qr(B @ rand_matrix)                              # long side by k
    Q.cuda(0)
    smaller_matrix = (Q.transpose(0, 1) @ B).cuda(0)             # k by short side
    U_hat, s, V = torch.svd(smaller_matrix,False)
    U_hat.cuda(0)
    U = (Q @ U_hat)
    
    if transpose:
        return V.transpose(0, 1), s, U.transpose(0, 1)
    else:
        return U, s, V

In [4]:
# computes an orthonormal matrix whose range approximates the range of A
# power_iteration_normalizer can be safe_sparse_dot (fast but unstable), LU (imbetween), or QR (slow but most accurate)
def randomized_range_finder(A, size, n_iter=5):
    Q = np.random.normal(size=(A.shape[1], size))
    
    for i in range(n_iter):
        Q, _ = linalg.lu(A @ Q, permute_l=True)
        Q, _ = linalg.lu(A.T @ Q, permute_l=True)
        
    Q, _ = linalg.qr(A @ Q, mode='economic')
    return Q

In [5]:
def randomized_svd(M, n_components, n_oversamples=10, n_iter=4):
    n_random = n_components + n_oversamples
    
    Q = torch.tensor(randomized_range_finder(M, n_random, n_iter)).cuda(0)
    # project M to the (k + p) dimensional space using the basis vectors
    M = torch.tensor(M).cuda(0)
    B = Q.transpose(0, 1) @ M
    # compute the SVD on the thin matrix: (k + p) wide
    Uhat, s, V = linalg.svd(B, full_matrices=False)
    Uhat = torch.tensor(Uhat).cuda(0)
    del B
    U = Q @ Uhat
    
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [6]:
def randomized_svd_original(M, n_components, n_oversamples=10, n_iter=4):
    
    n_random = n_components + n_oversamples
    
    Q = randomized_range_finder(M, n_random, n_iter)
    # project M to the (k + p) dimensional space using the basis vectors
    B = Q.T @ M
    # compute the SVD on the thin matrix: (k + p) wide
    Uhat, s, V = linalg.svd(B, full_matrices=False)
    del B
    U = Q @ Uhat
    
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [51]:
A = np.random.uniform(-40,40,[10,100]) 
randomized_svd(A, 5)

(tensor([[ 0.2665, -0.4784, -0.4189,  0.0461,  0.1967],
         [ 0.0450,  0.2502, -0.4644,  0.6517, -0.3875],
         [-0.0823,  0.5004,  0.1257, -0.2966, -0.1977],
         [ 0.2300,  0.0620, -0.3229, -0.6062, -0.0580],
         [ 0.4515,  0.2407,  0.3616,  0.1039,  0.1663],
         [ 0.5530,  0.1675, -0.0788, -0.0937, -0.4466],
         [ 0.0042,  0.1490, -0.4542, -0.2347,  0.0064],
         [ 0.1880, -0.5289,  0.1129, -0.1100, -0.3019],
         [ 0.1415,  0.2574, -0.2925,  0.0434,  0.6464],
         [-0.5497, -0.0077, -0.2174, -0.1706, -0.1805]], dtype=torch.float64, device='cuda:0'),
 array([ 291.1458,  272.6614,  258.4442,  243.4219,  235.0934]),
 array([[ 0.0518,  0.0131,  0.1641,  0.1506,  0.146 ,  0.0222, -0.0469,
          0.1355, -0.098 , -0.0233,  0.0092,  0.0286,  0.0281,  0.0301,
          0.0223,  0.0542,  0.0396,  0.0861,  0.1835,  0.17  , -0.104 ,
         -0.0503, -0.1279,  0.2209, -0.115 , -0.0889, -0.1002,  0.119 ,
          0.097 , -0.0717,  0.0485, -0.0142, -0

In [28]:
A = np.random.uniform(-40,40,[10,100]) 
simple_randomized_svd(A)

(array([[-0.0927, -0.3094,  0.0138, -0.498 ,  0.1362, -0.6804,  0.3705,
          0.0016, -0.1609, -0.0533],
        [-0.3696, -0.5729,  0.1055,  0.0333, -0.1139,  0.0144, -0.315 ,
          0.4539,  0.4073, -0.1962],
        [ 0.4726, -0.0089,  0.1479,  0.0591, -0.3718, -0.1061, -0.0042,
         -0.0511, -0.0456, -0.7727],
        [ 0.2914, -0.5239, -0.3223, -0.251 , -0.0146,  0.3509, -0.2546,
         -0.0093, -0.5257,  0.0952],
        [-0.1348,  0.3822,  0.1706, -0.4915,  0.2152,  0.3631,  0.0837,
          0.5217, -0.1876, -0.2691],
        [ 0.4823, -0.0967,  0.0848,  0.4299,  0.5133, -0.1813,  0.0785,
          0.5014, -0.0588,  0.093 ],
        [ 0.093 , -0.0529, -0.6686, -0.0841,  0.2823,  0.2049,  0.332 ,
         -0.0763,  0.4755, -0.2658],
        [-0.2028,  0.0905, -0.422 ,  0.2922, -0.5218, -0.0799,  0.3588,
          0.4363, -0.292 ,  0.0649],
        [ 0.496 ,  0.079 ,  0.0429, -0.3942, -0.4031, -0.0208,  0.0142,
          0.2405,  0.4233,  0.4363],
        [ 0.0036, -

In [29]:
A = np.random.uniform(-40,40,[10,100])
simple_randomized_torch_svd(A)

(tensor([[ 0.0022, -0.6062, -0.0613, -0.0791,  0.4002,  0.0024,  0.4948,
          -0.3578, -0.1446,  0.2620],
         [-0.4845, -0.1573, -0.2625,  0.5393,  0.0407,  0.3863, -0.0498,
          -0.1818,  0.0822, -0.4332],
         [ 0.1150,  0.0404,  0.2540, -0.4545,  0.3348,  0.1813, -0.1469,
          -0.3452,  0.5088, -0.4116],
         [-0.4244, -0.0384, -0.5109, -0.3065, -0.0551, -0.5866, -0.2308,
          -0.2045,  0.1428,  0.0278],
         [ 0.0904, -0.3397,  0.0358,  0.0862, -0.4325, -0.2028,  0.4178,
           0.2802,  0.5992, -0.1656],
         [ 0.0009, -0.3461,  0.3275, -0.0386,  0.0213, -0.3891, -0.1232,
           0.1403, -0.4726, -0.6011],
         [-0.5206,  0.2742,  0.4045,  0.1494,  0.4669, -0.2599,  0.2320,
           0.2895,  0.1736,  0.1205],
         [-0.1635, -0.5329,  0.2680,  0.0149, -0.0122,  0.1470, -0.6118,
           0.2005,  0.1841,  0.3836],
         [-0.1608,  0.1006,  0.5005,  0.1803, -0.4526, -0.1572,  0.0226,
          -0.6543, -0.0230,  0.1494],
 

In [7]:
np.set_printoptions(suppress=True, precision=4)

In [8]:
m_array = np.array([100, 1000, 20000])
n_array = np.array([100, 1000, 20000])

In [9]:
index = pd.MultiIndex.from_product([m_array, n_array], names=['# rows', '# cols'])

In [10]:
pd.options.display.float_format = '{:,.3f}'.format
df1 = pd.DataFrame(index=m_array, columns=n_array)
df2 = pd.DataFrame(index=m_array, columns=n_array)
df3 = pd.DataFrame(index=m_array, columns=n_array)
df4 = pd.DataFrame(index=m_array, columns=n_array)
df5 = pd.DataFrame(index=m_array, columns=n_array)
df6 = pd.DataFrame(index=m_array, columns=n_array)

In [11]:
### %%prun
for m in m_array:
    for n in n_array:      
        A = np.random.uniform(-40,40,[m,n])  
        t1 = timeit.timeit('simple_randomized_svd(A, 10)', number=3, globals=globals())
        t2 = timeit.timeit('decomposition.randomized_svd(A, 10)', number=3, globals=globals())
        t3 = timeit.timeit('fbpca.pca(A, 10)', number=3, globals=globals())
        t4 = timeit.timeit('simple_randomized_torch_svd(A, 10)', number=3, globals=globals())
        t5 = timeit.timeit('randomized_svd(A, 5)', number=3, globals=globals())
        t6 = timeit.timeit('randomized_svd_original(A, 5)', number=3, globals=globals())
        df1.set_value(m, n, t1)
        df2.set_value(m, n, t2)
        df3.set_value(m, n, t3)
        df4.set_value(m, n, t4)
        df5.set_value(m, n, t5)
        df6.set_value(m, n, t6)

/data/siavashmortezavi/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/data/siavashmortezavi/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/data/siavashmortezavi/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/data/siavashmortezavi/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/

In [12]:
df1/3

,100,1000,20000
100,0.007,0.021,0.018
1000,0.025,0.018,0.036
20000,0.014,0.042,0.364


In [13]:
df2/3

,100,1000,20000
100,0.059,0.092,0.094
1000,0.101,0.104,0.511
20000,0.124,0.483,4.159


In [14]:
df3/3

,100,1000,20000
100,0.024,0.077,0.084
1000,0.056,0.083,0.214
20000,0.090,0.225,1.424


In [15]:
df4/4

,100,1000,20000
100,1.202,0.014,0.027
1000,0.019,0.027,0.096
20000,0.029,0.100,2.756


In [16]:
df5/5

,100,1000,20000
100,0.030,0.050,0.056
1000,0.020,0.028,0.252
20000,0.050,0.238,2.225


In [17]:
df6/6

,100,1000,20000
100,0.026,0.043,0.046
1000,0.047,0.028,0.181
20000,0.046,0.173,1.198
